In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
from keras.models import load_model
from keras.models import Model
import cv2
import numpy as np
import tensorflow as tf
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.utils import to_categorical, plot_model
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

np.random.seed(1)


Using TensorFlow backend.


In [2]:
def check_units(y_true, y_pred):
    if y_pred.shape[1] != 1:
        y_pred = y_pred[:,1:2]
        y_true = y_true[:,1:2]
    return y_true, y_pred

def precision(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    y_true, y_pred = check_units(y_true, y_pred)
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def matthews_correlation(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [3]:
x_train = np.load("datasets/std/x_train_std_200.npy")
y_train = np.load("datasets/std/y_train_std_200.npy")
x_train.shape, y_train.shape

((22046, 200, 200, 3), (22046, 2))

In [4]:
x_val = np.load("datasets/std/x_val_std_200.npy")
y_val = np.load("datasets/std/y_val_std_200.npy")
x_val.shape, y_val.shape

((2756, 200, 200, 3), (2756, 2))

In [5]:
x_test = np.load("datasets/std/x_test_std_200.npy")
y_test = np.load("datasets/std/y_test_std_200.npy")
x_test.shape, y_test.shape

((2756, 200, 200, 3), (2756, 2))

In [6]:
img = x_train[50]
img.shape

(200, 200, 3)

In [7]:
print(x_train.shape, y_train.shape)
print(x_val.shape, x_test.shape, y_val.shape, y_test.shape)

(22046, 200, 200, 3) (22046, 2)
(2756, 200, 200, 3) (2756, 200, 200, 3) (2756, 2) (2756, 2)


In [8]:
IMAGE_SIZE = 200
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 20
BATCH_SIZE = 64
TEST_SIZE = 30
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [9]:
model = None 

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

In [10]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "binary_crossentropy", optimizer = opt,  metrics=['accuracy', precision, recall, f1, matthews_correlation, auc])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
__________

In [11]:
model.fit(x=x_train, y=y_train, validation_data = (x_val,y_val), batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1)

Train on 22046 samples, validate on 2756 samples
Epoch 1/20
22046/22046 [==============================] - 136s 6ms/step - loss: 0.3917 - acc: 0.8923 - precision: 0.9162 - recall: 0.8698 - f1: 0.8881 - matthews_correlation: 0.7893 - auc: 0.8803 - val_loss: 0.1827 - val_acc: 0.9307 - val_precision: 0.9866 - val_recall: 0.8735 - val_f1: 0.9258 - val_matthews_correlation: 0.8667 - val_auc: 0.9470
Epoch 2/20
22046/22046 [==============================] - 131s 6ms/step - loss: 0.1929 - acc: 0.9391 - precision: 0.9630 - recall: 0.9150 - f1: 0.9371 - matthews_correlation: 0.8802 - auc: 0.9579 - val_loss: 0.1578 - val_acc: 0.9358 - val_precision: 0.9874 - val_recall: 0.8843 - val_f1: 0.9321 - val_matthews_correlation: 0.8763 - val_auc: 0.9641
Epoch 3/20
22046/22046 [==============================] - 131s 6ms/step - loss: 0.1659 - acc: 0.9463 - precision: 0.9669 - recall: 0.9262 - f1: 0.9451 - matthews_correlation: 0.8942 - auc: 0.9680 - val_loss: 0.1445 - val_acc: 0.9445 - val_precision: 0.986

In [12]:
model.evaluate(x=x_test, y=y_test, verbose=1)

2756/2756 [==============================] - 8s 3ms/step


[0.1431749242113801,
 0.9637155297532656,
 0.9727437677854032,
 0.9529314330077483,
 0.9615423431071211,
 0.9277156786925561,
 0.9904584864746503]

In [13]:
def tensor_3to4(img):
    '''
    Resizes and expands dimension
    
    Input: img: a 3-channel image as input
    
    Returns a rank-4 tensor, since the network accepts batches of images
    One image corresponds to batch size of 1
    '''
    img = cv2.resize(img, (200, 200))
    img_4d = np.expand_dims(img, axis=0)  # rank 4 tensor for prediction
    return img_4d

In [14]:
model.layers

In [15]:
from keras.models import Model

layer_name = 'dense_2'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_layer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 100, 64)      36928     
__________

In [16]:
x_features = []
for x in x_train:
    x = tensor_3to4(x)
    intermediate_output = intermediate_layer_model.predict(x)
    x_features.append(intermediate_output)

In [17]:
len(x_features)

22046

In [18]:
x_features[55].shape

(1, 128)

In [19]:
x_features = np.array(x_features)
nsamples, nx, ny = x_features.shape
d2_train_dataset = x_features.reshape((nsamples,nx*ny))

In [20]:
results = []

In [21]:
classifier = SVC( probability=True, gamma=.1)
cv_score = cross_val_score(classifier, d2_train_dataset, y_train[:,1:2], cv=5, scoring='roc_auc')
results.append(cv_score)
classifier.fit(d2_train_dataset, y_train[:,1:2])

C:\Users\User\Anaconda3\envs\malaria\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\Anaconda3\envs\malaria\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\Anaconda3\envs\malaria\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\Anaconda3\envs\malaria\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
cv_score

array([0.99809686, 0.99817481, 0.99777932, 0.99855055, 0.99906393])

In [23]:
test_features = []
for y in x_test:
    y = tensor_3to4(y)
    intermediate_output = intermediate_layer_model.predict(y)
    test_features.append(intermediate_output)

In [24]:
test_features = np.array(test_features)
nsamples, nx, ny = test_features.shape
d2_test_dataset = test_features.reshape((nsamples,nx*ny))

In [25]:
y_pred = classifier.predict(d2_test_dataset)

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
accuracy_score(y_test[:,1:2], y_pred)

0.9241654571843251

In [28]:
from sklearn.metrics import classification_report

In [29]:
print(classification_report(y_test[:,1:2], y_pred))

              precision    recall  f1-score   support

         0.0       0.98      0.87      0.92      1409
         1.0       0.88      0.98      0.93      1347

   micro avg       0.92      0.92      0.92      2756
   macro avg       0.93      0.93      0.92      2756
weighted avg       0.93      0.92      0.92      2756



In [30]:
from sklearn import metrics

In [31]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In [32]:
matthews_corrcoef(y_test[:,1:2], y_pred)

0.8547848716915234

In [33]:
print("Done!")

Done!
